# Exploração de dados de soja usando a plataforma de acesso *Quandl*

Neste notebook explorarei algumas bases de dados que trazem informações sobre produção e preço de Soja (*soybeans*), especialmente no mercado futuro e de opções. Como neste primeiro momento desejo conhecer fontes potenciais de dados usarei a plataforma [*Quandl*](https://www.quandl.com/), que fornece acesso gratuito e facilitado a várias bases de dados financeiros. Os dados podem ser baixados diretamente da plataforma ou consultados via API. A *Quandl* também fornece o pacote *QuandlPython*, que pode ser instalado através do PIP:

In [ ]:
# !pip install quandl # uncomment for installing

Uma vez instalado o pacote deve ser carregado na sessão e, para acessar a API, o usuário precisa informar suas credenciais (obtidas a partir da sua conta no *Quandl*). 

In [2]:
import quandl as qd

In [5]:
qd.ApiConfig.api_key = "<sua API Key>"

In [6]:
# The code was removed by DSX for sharing.

## FAO - United Nations Food and Agriculture

A [FAO](https://www.quandl.com/data/UFAO-United-Nations-Food-and-Agriculture) disponibiliza um total de 33539 datasets sobre produtos agrícolas no mundo. Aqui estamos particularmente interessados na produção de soja (*soybeans*). Abaixo carregarei dois datasets contendo a produção brasileira e mundial, respectivamente. Cada dataset é uma série temporal com registros anuais, trazendo informações sobre:
* Toneladas de sementes (em toneladas);
* Área colhida (em hectares);
* Produtividade (em hectograma(100g)/hectare);
* Production Quantity (em toneladas).

Os dados oficiais da FAO são mais detalhados e também estão disponíveis gratuitamente em uma [plataforma própria](http://www.fao.org/faostat/en/#home).

In [39]:
ufao_cr_soyb_bra = qd.get("UFAO/CR_SOYB_BRA")
ufao_cr_soyb_bra.head()

,Seed - tonnes,Area Harvested - Ha,Yield - Hg/Ha,Production Quantity - tonnes
Date,,,,
1961-12-31,12545,240919,11268,271488
1962-12-31,13591,313640,11005,345175
1963-12-31,14384,339796,9503,322915
1964-12-31,17273,359622,8478,304897
1965-12-31,19627,431834,12115,523176


In [37]:
ufao_cr_soyb_5001 = qd.get("UFAO/CR_SOYB_5001")
ufao_cr_soyb_5001.tail()

,Seed - tonnes,Area Harvested - Ha,Yield - Hg/Ha,Production Quantity - tonnes
Date,,,,
2003-12-31,6535103,83609392,22801,190642011
2004-12-31,6551990,91526610,22454,205519398
2005-12-31,6710853,92437171,23178,214255282
2006-12-31,6310284,94937821,22986,218232719
2007-12-31,6791305,90199626,24449,220532612


Note que estas séries temporais somente trazem dados de 1961 até 2007, o que provavelmente é uma limitação para nossos objetivos:

In [69]:
s = "Brasil:\n  Ano max: {};\n  Ano min: {};\n\nMundial:\n  Ano max: {};\n  Ano min: {};".format( ufao_cr_soyb_bra.index.max().year,
                                                                               ufao_cr_soyb_bra.index.min().year,
                                                                               ufao_cr_soyb_5001.index.max().year,
                                                                               ufao_cr_soyb_5001.index.min().year )
print(s)

Brasil:
  Ano max: 2007;
  Ano min: 1961;

Mundial:
  Ano max: 2007;
  Ano min: 1961;


In [72]:
# remover os datasets da memória
try:
    del(ufao_cr_soyb_bra, ufao_cr_soyb_5001)
except NameError:
    pass

---

## CEPEA - Centro de Estudos Avançados em Economia Aplicada

O [CEPEA](https://www.cepea.esalq.usp.br/br) é um centro de pesquisas brasileiro focado em temas sobre a dinâmica da cadeia produtiva do agronegócio. É responsável também por monitorar o desempenho macroeconômico do setor, produzindo e disponibiliazando dados sobre o **PIB do Agronegócio e das cadeias produtivas** e **índices de exportação**. Dados oficiais de indicadores soja estão disponíveis para download [nesta](https://www.cepea.esalq.usp.br/br/indicador/soja.aspx) página, bem como as metodologias para sua contrução. O *Quandl*, entretanto, facilita o acesso a estes dados. A série temporal é construída por registros diários (dias úteis).

##### Indicador da soja ESALQ / BM&FBOVESPA - Paranaguá

In [73]:
cepea_soy_c = qd.get("CEPEA/SOYBEAN_C")
cepea_soy_c.tail()

,Price US$,Daily %,Monthly %
Date,,,
2017-11-09,22.79,0.49,2.75
2017-11-10,22.55,-1.05,1.67
2017-11-13,22.70,0.67,2.34
2017-11-14,22.39,-1.37,0.95
2017-11-16,22.46,0.31,1.26


##### Indicador da soja CEPEA/ESALQ - Paraná

In [74]:
cepea_soy = qd.get("CEPEA/SOYBEAN")
cepea_soy.tail()

,Price US$,Daily %,Monthly %
Date,,,
2017-11-09,21.18,0.43,2.87
2017-11-10,20.99,-0.90,1.94
2017-11-13,21.06,0.33,2.28
2017-11-14,20.92,-0.66,1.60
2017-11-16,21.10,0.86,2.48


Entretanto as colunas `Daily %` e `Monthly %` estão apenas guardadas para os últimos 15 dias:

In [31]:
d.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2912 entries, 2006-03-13 to 2017-11-14
Data columns (total 3 columns):
Price US$    2912 non-null float64
Daily %      15 non-null float64
Monthly %    15 non-null float64
dtypes: float64(3)
memory usage: 91.0 KB


---

## World Bank Global Economic Monitor (GEM) Commodities

Preços de commodities e índices desde 1960.



* [WGEC/WLD_SOYBEANS](https://www.quandl.com/data/WGEC/WLD_SOYBEANS-Soybeans-mt-current-World): Preço da soja mundial em dólar/mt. 

*obs*: mt = "metric ton".

In [35]:
d=qd.get("WGEC/WLD_SOYBEANS")
d.head()

,Value
Date,
1960-12-31,91.833333
1961-12-31,109.333333
1962-12-31,100.540000
1963-12-31,110.093333
1964-12-31,110.505833


---

## Banco Central do Brasil

Dados macroeconômicos brasileiros.

[BCB/4218](https://www.quandl.com/data/BCB/4218-Exports-Kg-Soybeans-including-grinded): Exportação ($kg$) de soja, incluindo triturada.

In [37]:
d=qd.get("BCB/4218")
d.head()

,Value
Date,
1989-01-31,5000000
1989-02-28,9750000
1989-03-31,10537650
1989-04-30,320065957
1989-05-31,373374202


[BCB/2971](https://www.quandl.com/data/BCB/2971-Exports-Fob-Soybeans-including-grinded): Exportação ($\$$) de soja, incluindo triturada.

In [39]:
d=qd.get("BCB/2971")
d.head()

,Value
Date,
1989-01-31,1817500
1989-02-28,3240425
1989-03-31,3169339
1989-04-30,91883451
1989-05-31,103203860


---

## Chicago Mercantile Exchange (CME)

Dados para uma grande variedade de commodities, incluindo grãos.

* [CME/SH1964](https://www.quandl.com/data/CME/SH1964-Soybeans-Futures-March-1964-SH1964-CBOT): Soybeans Futures, March 1964, SH1964, CBOT
* [CME/SH1963](https://www.quandl.com/data/CME/SH1963-Soybeans-Futures-March-1963-SH1963-CBOT): Soybeans Futures, March 1963, SH1963, CBOT
* [CME/SH1962](https://www.quandl.com/data/CME/SH1962-Soybeans-Futures-March-1962-SH1962-CBOT): Soybeans Futures, March 1962, SH1962, CBOT

In [34]:
d = qd.get("CME/SH1964")
d.head()

,Open,High,Low,Last,Change,Settle,Volume,Prev. Day Open Interest
Date,,,,,,,,
1963-04-02,250.0,251.2,250.0,None,None,250.2,0,0
1963-04-03,250.1,251.2,250.1,None,None,251.0,417,402
1963-04-04,251.2,251.2,250.2,None,None,250.2,156,498
1963-04-05,250.2,250.6,250.2,None,None,250.5,856,1158
1963-04-08,250.0,251.0,250.0,None,None,251.0,1011,2169


In [23]:
d = qd.get("CME/SH1963")
d.head()

,Open,High,Low,Last,Change,Settle,Volume,Prev. Day Open Interest
Date,,,,,,,,
1962-04-02,246.6,247.2,246.6,None,None,247.0,0,0
1962-04-03,246.6,246.6,246.6,None,None,246.6,17,17
1962-04-04,247.0,247.2,247.0,None,None,247.2,0,17
1962-04-05,247.2,247.6,247.2,None,None,247.4,10,27
1962-04-06,247.6,247.6,247.6,None,None,247.6,70,87


In [24]:
d = qd.get("CME/SH1962")
d.head()

,Open,High,Low,Last,Change,Settle,Volume,Prev. Day Open Interest
Date,,,,,,,,
1961-04-03,249.0,249.6,248.6,None,None,249.6,0,0
1961-04-04,249.6,250.4,249.4,None,None,250.2,507,492
1961-04-05,250.6,251.4,250.3,None,None,251.4,802,1254
1961-04-06,252.2,252.2,250.4,None,None,250.6,1036,1742
1961-04-07,251.4,251.4,249.6,None,None,250.2,1075,2573


---

## Wiki Commodities

In [33]:
d=qd.get("COM/WLD_SOYBEANS")
d.head()

,Column 1
Date,
1960-01-31,94
1960-02-29,91
1960-03-31,92
1960-04-30,93
1960-05-31,93
